In [1]:
import httpx
import pysolr

from xml.dom.minidom import parse
import xml.etree.ElementTree as etree
from Marc_to_Bibframe.Marc.marcWork import MarcWork
from Marc_to_Bibframe.Marc.marcInstance import MarcInstance
from Marc_to_Bibframe.Marc.marcItems import MarcItems
from Solr.solr import create_doc

In [2]:
#SOLR
solr = pysolr.Solr('http://localhost:8983/solr/search/', timeout=10)
solr.ping()

'{\n  "responseHeader":{\n    "zkConnected":null,\n    "status":0,\n    "QTime":17,\n    "params":{\n      "q":"{!lucene}*:*",\n      "distrib":"false",\n      "df":"_text_",\n      "rows":"10",\n      "echoParams":"all",\n      "rid":"localhost-78"}},\n  "status":"OK"}\n'

DELETE ALL DOCS

In [3]:
solr.delete(q="*:*", commit=True)

'<?xml version="1.0" encoding="UTF-8"?>\n<response>\n\n<lst name="responseHeader">\n  <int name="status">0</int>\n  <int name="QTime">1343</int>\n</lst>\n</response>\n'

ADD DOCS

In [3]:
def UpadateSolr(path_marc, count, shelf, out):
    
    marc_file = parse(path_marc)
    records = marc_file.getElementsByTagName('record')

    docs = list()
    for record in records:
        
        marcxml = etree.fromstring(record.toxml())
        workMarc = MarcWork(marcxml)
        instanceMarc = MarcInstance(marcxml)
        itemsMarc = MarcItems(marcxml)
        print(records.index(record))
        print(workMarc.Title().get('title'))

        #SOLR
        doc = create_doc(workMarc, instanceMarc, itemsMarc, 'Livro', shelf, count)
        docs.append(doc)

        count += 1

    return docs

In [8]:
count = 20
e = '1'
p = '2'
out = f'E{e}/P{p}'
path_marc = f'Input/{out}/koha.xml'

docs = UpadateSolr(path_marc, count, f"E{e}.P{p}", out)

0
Resumos
1
Resumos
2
Pesquisa científica e tecnológica em saúde
3
Pesquisas recentes em energia, meio ambiente e tecnologia
4
Iniciação à pesquisa científica
5
<1º >Seminário de Ciências da Fiube
6
Encontro sobre o programa de pesquisas para o trópico úmido.
7
A pesquisa no Brasil
8
Resumos do XVII Seminário de Iniciação Científica da UFPA
9
FADESP, Fundação de Amaparo e Desenvolvimento da Pesquisa, Belém-Pará.
10
Manual de normas para publicações técnico-científicas
11
Manual do autor
12
A estatística na pesquisa científica
13
Prioridades da pesquisa do CIFOR
14
Matemática e estatística na análise de experimentos e no melhoramento genético
15
Relatório de atividades 2011-2013
16
Seminário pesquisa tecnológica
17
Pesquisa científica
18
Pesquisa operacional
19
Ciência, tecnologia e inovação para um Brasil competitivo
20
Os 40 anos da fundação Ford no Brasil
21
Orçamento da União para ciência e tecnologia, 1980
22
Guia de fontes de financiamento à ciência & tecnologia
23
Guia de fontes 

In [7]:
docs[0]

{'id': 19,
 'call': '001.4 M916',
 'shelf': 'E1.P2',
 'title': 'Resumos',
 'responsibilities': 'I Mostra Técnico-Científica da Universidade do Amazonas, Manaus, AM, 1999.',
 'place': 'Manaus',
 'publisher': 'UA',
 'subject': ['Ciência--Amazonas--Congressos',
  'Tecnologia--Amazonas--Congressos'],
 'type': 'Livro',
 'items': ['05-0436'],
 'author': None,
 'year': '1999'}

In [9]:
solr.add(docs, commit=True)

'{\n  "responseHeader":{\n    "status":0,\n    "QTime":699}}\n'